For this notebook, I am going to be going through the titanic dataset given on Kaggle: https://www.kaggle.com/c/titanic, processing the training data with Linear and Logistic Regression, using those models to predict survivor rates on the test set and finally submitting it to the website. 

![alt text](http://www.globalnerdy.com/wordpress/wp-content/uploads/2015/08/if-the-titanic-sank-today1.jpg "If the titanic sank today")

<h1>Training</h1>

<h2>Linear Regression</h2>


In [3]:
import pandas as pd 
titanic = pd.read_csv("features/train_features.csv")


In [4]:
print titanic.head(5)


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex  Age  SibSp  \
0                            Braund, Mr. Owen Harris    male   22      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   38      1   
2                             Heikkinen, Miss. Laina  female   26      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   35      1   
4                           Allen, Mr. William Henry    male   35      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [5]:
print titanic.describe()

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


First we see that Age is missing a few entries. There are 714 instead of 891 like the other columns have. Lets fill the missing entires with the median on the age column


In [6]:
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
print titanic.describe()

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.361582    0.523008   
std     257.353842    0.486592    0.836071   13.019697    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   22.000000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   35.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


Now lets find a way to convert the Sex and Embarked columns to numeric values so that they can be trained on. 

In [7]:
titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic.loc[titanic["Sex"] == "female","Sex"] = 1

titanic["Embarked"] = titanic["Embarked"].fillna("S")  #similar to above, we ensure that the embarked column has an appropriate matching number of entries

titanic.loc[titanic["Embarked"] == "S","Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C","Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q","Embarked"] = 2

Now that our dataset is in order, we are ready to begin learning on it. Remember, the goal is to predict the survivor column correctly. In short, we want to use the other features to predict whether this particular passenger was killed (0) or survived(1)- heres hoping. First of all, lets try a simple linear regression

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold     # this is what we will be using to split the data

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]   # these are our features

model = LinearRegression()   

kf = KFold(titanic.shape[0], n_folds= 3, random_state=1)   #splitting folds into train (594 items) and test (297) groups in 3 different random ways


In [28]:
predictions = []

for train, test in kf:   
     
    
    train_predictors = (titanic[predictors].iloc[train,:])   # only take single row. this creates the feature vector
    
    train_target = titanic["Survived"].iloc[train]  #only the single label for each feature vector 
    
    model.fit(train_predictors,train_target)  #fits a linear regression to this result. 
    
    test_predictions = model.predict(titanic[predictors].iloc[test,:])  #the models prediction of the label based on the test features, not the train features
    predictions.append(test_predictions)
    
print len(predictions),len(predictions[0])   #this should be 3,297 - 3 sets of 1 prediction for each test feature vector 

3 297


Now lets evaluate our error metric. We know from the competition rules that the error metric is the percentage of correct predictions, pretty simple :). For this we need the number of values in predictions that is the same as titanic["Survived"] - the true value, as well as the total number of passengers. 

First of all, lets combine our predictions into a single array with length 891. 

In [29]:
import numpy as np

predictions = np.concatenate(predictions, axis = 0)
print predictions.shape

(891,)


Now lets map predictions to outcomes. And then we can compute our accuracy with one simple line 

In [33]:
predictions[predictions > .5] =1 
predictions[predictions <= .5] =0      # basically set to 1 if its greater than .5, set to 0 if its less than or equal to

accuracy = sum(predictions[predictions == titanic["Survived"]]) / len(predictions)
print accuracy

0.783389450056


/Users/alex/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


Not bad! 78% accuracy. I think its impressive, but apparently the machine learning community is shooting for much higher :p Next Lets try Logistic Regression

<h2>Logistic Regression</h2>

In [37]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(random_state=1)

scores = cross_validation.cross_val_score(log_model,titanic[predictors],titanic["Survived"],cv=3)  #because we already created the predictors numpy array above, we can re-use it. All logistic regression does is replace or rudimentary mapping with a logit function for mapping values to 0 or 1

print scores.mean()  #scores is created with 3 seperate instances because the cross validation is done thrice

0.787878787879


Well thats not much of an improvement, but all we did really is change our activation function, so not much can be expected :o

<h1>Testing</h1>

Remember this model cannot be expected to predict any passengers survival rate with 78% accuracy. That is just our cross validation accuracy. To evaluate its true usefulness, we need to use test data. Additionally, the compeition will score us based on our accuracy with test data only, so lets do that. 

We will need to process the dataset in the same way, converting any character columns to numeric ones so they can be trained on, and ensuring that all columns have the same number of entries in them (Im looking at you Embarked!!!! >:(



In [40]:
titanic_test = pd.read_csv("features/test_features.csv")

#filling in the missing values
titanic_test["Age"] = titanic_test["Age"].fillna(titanic["Age"].median())
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")
titanic_test["Fare"] = titanic_test["Fare"].fillna(35)    #filling the missing Fare column with the mean fare

#converting character columns into numeric ones
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0 
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test.loc[titanic_test["Embarked"] =="S","Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] =="C","Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] =="Q","Embarked"] = 2

print titanic_test.describe()

       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  418.000000  418.000000  418.000000  418.000000
mean   1100.500000    2.265550   29.805024    0.447368    0.392344   35.625688
std     120.810458    0.841838   12.667969    0.896760    0.981429   55.840509
min     892.000000    1.000000    0.170000    0.000000    0.000000    0.000000
25%     996.250000    1.000000   23.000000    0.000000    0.000000    7.895800
50%    1100.500000    3.000000   28.000000    0.000000    0.000000   14.454200
75%    1204.750000    3.000000   35.750000    1.000000    0.000000   31.500000
max    1309.000000    3.000000   76.000000    8.000000    9.000000  512.329200


Great! we have everything we need for the submission fil. All our columns have the same amount of data and are all numeric values. Now, lets use our logistic model created earlier to predict Survivor rates for this group of test passengers


In [45]:
log_model.fit(titanic[predictors],titanic["Survived"])

test_predictions = log_model.predict(titanic_test[predictors])  #same predictor columns as before, this time with the test data

submission = pd.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": test_predictions
    })
    
submission.to_csv("kaggle.csv", index=False)
